<a href="https://colab.research.google.com/github/phyop/220528_Colab/blob/main/220519_1211_BERT%E9%96%8B%E6%BA%90%E9%A0%85%E7%9B%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CNN有10幾層，但RNN通常就一兩層
# RNN的輸入，除了自己的X(n)之外，還會用到前一個的中間結果h(n-1)
# 因為要等前一個輸出，所以RNN的缺點就是沒辦法平行計算

"""
《並發、並行、同步、阻塞》
# 並發（complicating）是你可中斷，先做別的事，之後再回來繼續做
# 並行（parallel）是同時一起做

你吃飯吃到一半，電話來了，你一直到吃完了以後才去接，這就說明你不支持並發也不支持並行。
你吃飯吃到一半，電話來了，你停了下來接了電話，接完後繼續吃飯，這說明你支持並發（complicating）。
你吃飯吃到一半，電話來了，你一邊打電話一邊吃飯，這說明你支持並行（parallel）。

# 同步（Synchronous）是，要自己主動去看的
# 異步( Asynchronous)是，會被通知做完的
# 人家還沒做完，自己就什麼也不能做的狀態，就是阻塞( Blocking )
# 人家還沒做完，自己可以先做別的事的狀態，就是非阻塞( Nonblocking)

老張把水壺放到火上，立等水開。（同步阻塞）
老張把水壺放到火上，去客廳看電視，時不時去廚房看看水開沒有。（同步非阻塞）
老張還是覺得自己有點傻，於是變高端了，買了把會響笛的那種水壺。水開之後，能大聲發出嘀~~~~的噪音。
老張把響水壺放到火上，立等水開。（異步阻塞）
老張把響水壺放到火上，去客廳看電視，水壺響之前不再去看它了，響了再去拿壺。（異步非阻塞）

In [ ]:
# X(t)是中間輸入，h(t)是中間輸出，向右傳給下一個的，是中間隱層特徵
# RNN不能並行，所以transformer的出現就是爲了解決這件事
# 有些詞重要，大部分的詞只是描述，所以就出現了attention
# 只要把self-attention的機制搞懂了，那transformer就差不多了
# RNN的結構圖是一個傳一個的，而觀察transformer的結構圖
# 在encoder中，是多個詞向量x(n)一次進入self-attention
# 各轉成z(n)向量，然後再一起進到feed forward，然後對各詞向量做輸出
# 這就是transformer的self-attention，在網路結構上，對RNN做的加強

In [ ]:
# 不同的詞，在不同的語境當中，會有差異，這是在NLP常見的一種任務
# 比如「幹哈那」在東北哈爾濱，不是真關心你在幹嘛，而是看有沒有空，一起出去喝兩杯
# 又比如說，類似的一句話，it有時候是指生命體，有時候是指非生命體
# The [animal] didn't cross the street because it was too tired.
# The animal didn't cross the [street] because it was too naroow.
# 傳統word2vec，預訓練好的向量，就不再變了，所以無法區別出這種語意
# attention的熱度圖就是，對圖像的特征提取，把哪些部分當做重點了
# 也就是對這樣的圖像數據，有看到什麼特點，是可以做為分辨能力使用的 
# 同理我們也期待，一句話進去，可以對其做熱度圖，分辨出來哪個詞是重點，多關注一點
# 傳統word2vec，每個詞的權重是一樣的。而self-attention，是有熱度圖的
# 也就是會像圖像那樣，把self上下文一起抓進來看，而有熱度圖attention
# 所以transformer中，不止轉成詞向量，還多增加了attention的權重

In [ ]:
# 比如一句話裡面，就只有Thinking Machines這兩個詞
# 在第1個詞看來，上下文這兩個詞，分別對我做什麼樣的貢獻
# 在第2個詞看來，上下文這兩個詞，分別對我做什麼樣的貢獻
# 然後再對原本的詞向量，調整成新的編碼
# Transformer創建了3個輔助矩陣計算權重，queries、keys、values
# 來查詢當前詞與其他詞的關係，以及特征向量的表達
# self-attention要訓練的3個矩陣，所以就得到
# 要去查詢的 query(q1、q2) 
# 等着被查的 key(k1、k2)
# 實際的特徵信息 value(v1、v2)
# 想像一句話有2個詞，所以詞向量矩陣X，有2個row，分別是x1、x2
# 假設詞向量特徵維度是4，那權重矩陣WQ，就是4 row × N col
# 句子矩陣X，乘上權重矩陣WQ後，得到矩陣Q，是2 row × 3 col
# WQ.shape = (詞向量維度, 權重維度)
# Q.shape = (句子長度, 權重維度)

In [ ]:
# 內積的特性是，如果2特徵之間無關的話，其內積值爲0
# x1軸和x2軸之間如果是垂直，代表2者無關，所以內積就是0
# x1軸和x2軸之間夾角theta，如果越小，代表2者相關度越大，所以內積值越大
# 所以計算q和k之間的關系，也可以使用內積值，來看關聯性程度
# 內積就是，同樣的矩陣位置相乘
# softmax就是，讓x值先經過e^x來放大，然後再做normalize歸一化
# 可以利用softmax，來決定當前詞，對編碼這個位置的影響程度
# 把q × k^T的結果，傳進去softmax，就可以得到影響概率

In [ ]:
# 以第一個詞的觀點，利用q1去查句子中各個詞k(n)
# 每個詞的Q,會跟整個序列(句子)中,每一個k(詞)計算得分
# 第一個詞，和同序列別詞的關聯度 ->（q1·k1, q1·k2, ..., q1·kn）-> （要去查詢的q1）·（等着被查的kn）
# 內積小的,關聯度小,就代表不重要,不需要關注
# 每個詞都有自己的，實際的特徵信息v(n)
# 根據計算的關注度得分,也就是權重,再乘上該詞的特徵
# v1,v2...vn，的維度都是相同的
# 因爲向量維度越長，相乘的東西多了，內積值自然越大，那就會顯得好像越重要
# 這就不太合理了，所以歸一化的方式就是，把結果值除以向量維度的根號
# （q1·k1, q1·k2, ..., q1·kn）->（112, 96）-> 假設維度是64 ->（14,8）->（0.88, 0.12）
# 所以對第一個詞，重新做編碼成 z1 = 0.88v1 + 0.12v2
# 所以,q1如果對到第n1個詞,是大的權重,那q1就包含第n1詞的特徵多一些
# q1如果對到第n2個詞,內積根本爲0,代表完全無關聯,那q1就完全不需要去包含第n2詞的特徵了
# 目前只用q1當作舉例，去對句子中其他詞做關聯性
# 同理，第n個詞，也會去對句子中的其他詞，做關聯性
# 乍想起來，會是用for循環去一個個做，但這也是一次次有序性的等待，不是並行計算
# 因爲一個詞，就是一row的值，那其實就可以把所有的詞向量，所有的qkv，都以一個詞一row，疊成矩陣，幾個矩陣相乘，就可以並行算完一個序列了
# 並行的計算方式，大大提升計算效率，這就和傳統word2vec，用RNN、LSTM，要用for慢慢移動序列窗格做計算，有很大的區別了

In [ ]:
# 這樣，自己句子中的詞，大家之間的相關度，就是self-attention
# 所以做完self-attention之後,每一個詞它的向量,就不再只代表是自身了
# 而是把序列(句子)全部詞,都摻雜考慮之間的相關性了
# 傳統word2vec,只考慮一個詞的「出現位置」,通常會在怎樣的前後文之中
# word2vec有種，隨着窗格的大小，出現的位置越近，得分值越高的感覺
# 而self-attention,不是考慮一個詞的出現位置,而是序列中的詞,其「互相關聯度」
# 所以self-attention，不是出現位置越近得越高。這和傳統word2vec就有很大的區別了

In [ ]:
# 一個詞就有一個q，一個k，一個v，這就類似CNN裏面，一個卷積核，生出一張特徵圖
# 而我們知道，特徵越多，辨識度越好。所以同理，有越多組qkv的話，就類似越多卷積核，所以可能可以提取到更多種特徵
# 卷積核可能有幾十個，但transformer的multi-headed，通常只有8軸qkv
# 做完這8組，把特徵都拼接起來，一個詞的特徵就會好長
# 所以再把這一長串特徵，通過全連接去做降維，比如1024維降到128維這樣
# 最後就有一種，濃縮了多種食材，把中間過渡的多餘水費給蒸發去除掉，留下精華的感覺
# 所以，multi-headed就類似CNN中的多個卷積核


In [ ]:
# 如果把注意力畫成熱度圖，會發現一個詞跟自己附近的詞，注意力是比較大的
# 這個現象和word2vec的原理是不謀而合的，所以transformer有這樣的現象，就讓人對這個架構，比較有了信心
# CNN的多層卷積，是爲了從粗特徵中，再提取特徵，得到更細致的特徵。同理也可以應用到transformer
# 所以那我們就把這樣的結構，給個名字叫做encoder，仿造多層卷積那樣，給它多次應用下去
# 此外，LSTM可以選擇輸出是向量的形式，這樣就可以再次投入LSTM
# 而encoder的輸出r1、r2（舉例序列中只有2個詞的話），本來就是個向量，所以也可以像是LSTM，直接多層接下去
# 也就是encoder的輸出，可以當下一個encoder的輸入

In [ ]:
# encoder包含了2大部分，attention、feed forward
# feed forward是将Multi-Head Attention得到的提炼好的向量，再投影到一个更大的空间（论文里将空间放大了4倍）
# 在那个大空间里可以更方便地提取需要的信息（使用Relu激活函数），最后再投影回token向量原来的空间
# decoder包含了3大部分，masked、attention、feed forward
# https://zhuanlan.zhihu.com/p/47510705
# encoder後面直接傳到decoder，叫做encoder-decoder attention
# decoder後半部分，其實就是encoder
# 也就是，decoder只是比encoder多了一次attention而已
# 而且decoder多的attention，和encoder略有不同，是masked
# 此外，在encoder、decoder中的所有動作，後面都會接「add & norm」
# 這就是重新編碼步驟，例如z1 = 0.88v1 + 0.12v2
# norm，算出那個詞的相關度，然後做歸一化
# add，把序列中其他詞，都考慮進來相加：[那個詞的相關度（歸一化後）] × [那個詞的特徵值]
# multi-head attention，就是設置多組qkv，次數以Nx來表示，寫在encoder、decoder的外側
# linear，將多組qkv得到的特徵都concat起來，然後再用全連接dense去對維度降維

In [ ]:
# encoder，self-attention，去對每個詞，以k和v表示成新的編碼
# decoder，會透過q去查encoder，得到的key是什麼，特徵向量表達是什麼，做綜合利用
# decoder，用mask去遮蔽後面的詞，只根據前面看得到的詞，來預測後面的詞
# 和encoder一次看到序列全部的詞不同，decoder是一次得到一個結果，然後移動mask，基於前面的結果，再得到一個下一次的結果
# decoder輸出特徵，連到softmax做分類任務，也就是在語料庫中，找那個詞最適合
# 既然都已經要用BERT，那用人家的預訓練的結果做微調就好，不用自己做隨機初始化，從頭開始
# positional encoding，周期性訊號，做位置編碼，也就是k1...kn和v1...vn的線性組合

In [ ]:
# 隨着層數的堆疊，可能會越來越見樹不見林，越學越差，所以可以用殘差網路的方式去解決
# 一條路原封不動同等映射拿過去，另一條路自己學去，哪個學得好學哪個，至少不必原來的差
# 殘差網路方式，會讓「add&norm」會有2個輸入，分別是來自前一步處理後的輸出數據，以及前一步處理前的原始輸入數據
# encoder端的輸出，就是讓每個詞都有自己的特徵編碼r1...rn
# transformer是得到特徵的表達，BERT則是把transformer拿到的特徵，去做訓練
# BERT只需要知道，當前輸入的特徵表達是什麼就好了，所以只要是transformer當中的encoder就夠了
# NLP最難的部分，就是對特徵做編碼。分類任務是一點也不難，其實
# 說白了，BERT就是transformer當中的encoder部分
# BERT，Bidirectional Encoder Representations from Transformers
# BERT是開源的，提供了很多預訓練好的模型，也就是最麻煩的部分，不用自己去玩了。會修改來做應用就行了
# BERT訓練的參數，就是encoder中的qkv權重矩陣
# BERT的損失函數就是，如果可以把mask掉的詞預測出來，那就代表對上下文的重點是理解正確的
# CBOW或是skip-gram，都沒有用到標籤，BERT也不需要
# 比如「今天 [mask] 玩 DOTA」，是把第2個詞給mask掉，是一個從語料庫求詞的多分類任務 
# 英文的詞和字是一樣的，但中文的詞就比字多太多了，所以雖然應該用詞比較好，但從字下手，可能才比較實際
# 訓練的方法一，比如可以隨機把15%的詞給mask掉
# 訓練的方法二，比如利用分類字符[cls]判斷，做一個多分類任務yes/no，看兩個句子是否應該連在一起
# 字符也要做成編碼，[seq]是兩個句子之前的連接符；[cls]是要做分類的向量
# 不做位置編碼的情況下，分類字符放哪裏都一樣
# 因爲要做dot，所以start、end的輔助向量，維度必須要和詞向量的維度匹配，匹配通常就是同維度的意思
# 對每個詞做dot的結果，經過softmax後，就從概率值最大的位置，得到了文章閱讀題，答案的start、end位置了

In [ ]:
# 在Github中，直接選星星數最多的，比如搜尋BERT，最多星的就是官方的google-search
# 在Github中，如果有一堆檔案，那就先略過，直接看說明，說明中則直接看有連結的部分
# 藍色連結部分，可能是可以下載的預訓練模型，比如想要把英文全部轉成小寫，那就選uncased
# 其他還可區分成：不同的層數（參數大小的差異）、訓練資料來源（英文、中文）、大小寫、混合語言...
# 其中的json檔，是個像是字典的結構，存放一些比如網路結構等參數資訊，是在訓練過程中需要用到的
# checkpoint 檔 (.ckpt)，簡單的說它儲存了變量 (variable) 的名字和對應的張量 (tensor) 數值，所以檔案會會比較大
# 當你建立了一個 Saver 物件以後，是可以選擇每個 variable 要用什麼名字儲存在 checkpoint 檔裡的．預設上則是會使用 Variable.name 的值
# vocab.txt則是語料庫會用到的詞，也就是這些詞我們有給他們ID編號

In [ ]:
# 模型文件放到BERT_BASE_DIR，data文件放到glue_data當中
# data就是，官方提供一些可以練手的小項目，比如MRPC的數據量只有3600條，比較少，連沒有GPU都夠用
# 一筆資料一行，行首是標籤0/1（2分類任務），看這兩句意思是否相同。後面接的是data，包含有2個ID，分別對應到2句話
# 測試集裏面比訓練集少了標籤0/1
# 按下右上角綠色的clone or download，把GitHub上的所有項目資料下載下來
# 在IDE中創建新project，然後把這些項目文件全部放進來這個projet中
# 基於自己想要做的不同任務，自己去選擇要fine tune哪個模型，比如MRPC是sentence classification tasks，那網路架構就去下載uncased_L-12_H-768_A-12

In [ ]:
# GLUE 列表包含 9 個句子級別的分類任務，分別是：
# CoLA（語言可接受性語料庫）識別句子在語法上是否正確
# Mnli（多基因自然語言影響）給出一個假設並判斷另一個句子與假設之間的關係：條目、合同或無關。
# MRPC（Microsoft Research Paraphrase Corpus）判斷兩個句子是否是相互的釋義
# Qnli（問答自然語言推理）判斷第二句是否包含第一個問題的答案。
# QQP（Quora Question Pairs2）判斷兩個問題是否具有相同的語義。
# RTE（識別文本條目）確定一個句子是否與假設有蘊含關係。
# Sst-2 (Stanford sentient treebank) 判斷一個句子的正負情緒
# STS-B（Semantic Textual Similarity Benchmark）判斷兩個句子的相似度（分數為1-5）。
# Wnli（Winograd 自然語言影響）判斷一個帶有匿名代詞的句子和一個帶有代詞替換的句子是否包含

In [ ]:
《conda安裝套件》
$ conda activate /home/sean-m164/anaconda3/envs/220510_BERT

$ /home/sean-m164/anaconda3/bin/conda install -p /home/sean-m164/anaconda3/envs/220510_BERT tensorflow -y

"""
# https://docs.conda.io/projects/conda/en/latest/commands/install.html
-p, --prefix
Full path to environment location (i.e. prefix).
-y, --yes
Do not ask for confirmation.
"""

《用txt安裝指定的套件》
pip install -r requirements.txt

In [ ]:
《對py檔按右鍵 -> （run下面）運行配置 -> 形參 -> 插入宏 -> 剪貼板內容》

# 在當前目錄下，執行「Python」這個指令。如果沒有設爲環境變數，那就得去執行檔放置的位置才能呼叫成功
# 「Python」這個指令，後面接的第一個非字典參數，則是目標檔案
# 再後面的參數，因爲不是按照可放置參數的順序，所以是以字典形式給予
python run_classifier.py \ 
--task_name=MRPC \ # 換行不結束
--do_train=true \ # 是否要訓練
--do_eval=true \ # 是否要驗證，比如準確率
# 本來是「--data_dir=$GLUE_DIR/MRPC」，$的意思就是個人的環境變數，也就是要把$的地方，改成我們自己的GLUE位置，然後在下面建個「/MRPC」
# 絕對路徑是 /home/sean-m164/PycharmProjects/220510_BERT/GLUE/glue_data/MRPC
# 盡量不要用絕對路徑，因爲一個項目是一包的，所以如果項目做移動，裏面檔案間的相對位置也不會改變
--data_dir=../GLUE/glue_data/MRPC \ 
# github上說明檔的原值是 $BERT_BASE_DIR/vocab.txt
--vocab_file=../GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/vocab.txt \ # 語料表
# github上說明檔的原值是 $BERT_BASE_DIR/bert_config.json
--bert_config_file=../GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/bert_config.json \ # json的內容是字典形式，所以json檔適合拿來放網路架構配置
# checkpoint檔是專門拿來放，數值量大的tf張量資料
# github上說明檔的原值是 $BERT_BASE_DIR/bert_model.ckpt
--init_checkpoint=../GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/bert_model.ckpt \ # 把人家預訓練好的權重加載進來
--max_seq_length=128 \ # 每句話最長的長度。也就是時間序列窗格的大小
# github上說明檔的原值是32，對一般的個人電腦來說有點大，所以改成8就好
# 如果batch_size=8都還被報錯，那就再調小一點，比如乾脆給它設1就好，反正學code而已嘛，只要跑得動就好了
# 因爲BERT這個模型，比較重量級，也就是需要的資源比較大
# 我的筆電設定8的時候，batch就會顯示None。數值往下實驗，最多跑到batch_size=7,還可以顯示的出來，9也顯示不出來，所以乾脆就設個4就好
# 如果後來就算調batch_size=1也顯示None。那就epochs隨便改個5之類的，然後batch_size=8，再去全部重跑一次
# 重跑完之後，去debug模式，batch_size調成1，應該就不會顯示None，而是1了
# 這時候再一步步往上加，比如調到2,再調到4。batch_size就有可能都會顯示出來了
# 直到值調的太高，就又會顯示成None。那就要再隨便動個其它運算參數，重跑一次，再去debug模式，batch_size調成1->2->4->...
--train_batch_size=4 \ 
--learning_rate=2e-5 \ # 學習率好像都e-5這個等級之類的
--num_train_epochs=3.0 \ # 個人練習不追求好的準確率，epoch選1～3就夠了
# github上說明檔的原值是 /tmp/mrpc_output/
# 資料夾的位置，最好都先自己創建出來，以免中間過程發生錯誤，浪費一堆時間debug，重來一次
--output_dir=../GLUE/output # 保存訓練的模型
 

In [ ]:
--task_name=MRPC \ 
--do_train=true \ 
--do_eval=true \ 
--data_dir=../GLUE/glue_data/MRPC \ 
--vocab_file=../GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/vocab.txt \ 
--bert_config_file=../GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/bert_config.json \ 
--init_checkpoint=../GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/bert_model.ckpt \ 
--max_seq_length=128 \ 
--train_batch_size=8 \ 
--learning_rate=2e-5 \ 
--num_train_epochs=5.0 \ 
--output_dir=../GLUE/output 

In [ ]:
《run_classifier.py》

def main(_):
  ...
  if FLAGS.do_train:
    ...
    # 這邊就是會列印出我們給形參train_batch_size的地方
    tf.compat.v1.logging.info("  Batch size = %d", FLAGS.train_batch_size)

In [ ]:
《module 'tensorflow._api.v2.train' has no attribute 'Optimizer'》
https://stackoverflow.com/questions/61250311/error-importing-bert-module-tensorflow-api-v2-train-has-no-attribute-optimi

對bert-master/optimization.py 嘗試更改第 87 行的代碼
class AdamWeightDecayOptimizer(tf.train.Optimizer):

# （方法1）change to
class AdamWeightDecayOptimizer(tf.compat.v1.train.Optimizer):
    
# （方法2）change to
class AdamWeightDecayOptimizer(tf.keras.optimizers.Optimizer):

In [ ]:
《module 'tensorflow' has no attribute 'flags' 》
https://github.com/tensorflow/tensor2tensor/issues/1754
tensorflow2.0中，tf.flags.FLAGS好像已經移到tf.compat.v1.flags.Flag

# tf.compat.v1.flags.Flag
https://www.tensorflow.org/api_docs/python/tf/compat/v1/flags/Flag

# tf.compat.v1.flags.DEFINE_string
https://www.tensorflow.org/api_docs/python/tf/compat/v1/flags/DEFINE_string

In [ ]:
《BERT的一堆坑》
https://abecode.github.io/nlp/2020/12/10/first-tries-with-bert.html

In [ ]:
《tf.gfile.Open() -> tf.io.gfile.GFile()》
https://www.tensorflow.org/api_docs/python/tf/io/gfile/GFile

with open("/tmp/x", "w") as f:
with tf.io.gfile.GFile("/tmp/x") as f:

In [ ]:
 《tf.python_io.TFRecordWriter() -> tf.compat.v1.python_io.TFRecordWriter()》
 # writer = tf.python_io.TFRecordWriter(output_file)
  writer = tf.compat.v1.python_io.TFRecordWriter(output_file)

In [ ]:
《tf.logging -> tf.compat.v1.logging》
tf_upgrade_v2將升級腳本和更改tf.logging為tf.compat.v1.logging

In [ ]:
tf.FixedLenFeature -> tf.io.FixedLenFeature

In [ ]:
# TensorFlow2.x 和 TensorFlow1.x 的函数转换方法
https://zhuanlan.zhihu.com/p/495830763

In [ ]:
# tf.compat.v1.estimator.data.map_and_batch() -> tf.data.experimental.map_and_batch()
https://www.tensorflow.org/api_docs/python/tf/data/experimental/map_and_batch

https://runebook.dev/zh-CN/docs/tensorflow/data/experimental/map_and_batch
tf.compat.v1.data.experimental.map_and_batch 警告：此功能已被弃用。它将在将来的版本中删除。

In [ ]:
# tf.parse_single_example() -> tf.io.parse_single_example()

In [ ]:
# tf.to_int32() -> tf.compat.v1.to_int32()

In [ ]:
# tf.variable_scope() -> tf.compat.v1.variable_scope()

In [ ]:
# tf.get_variable() -> tf.compat.v1.get_variable()

In [ ]:
# tf.truncated_normal_initializer() -> tf.compat.v1.truncated_normal_initializer()

In [ ]:
# tf.assert_less_equal() -> tf.compat.v1.assert_less_equal()

In [ ]:
《tf.contrib》
# tf.contrib -> tf.compat.v1.estimator
# tensorflow.contrib.layers -> tf_slim
https://blog.csdn.net/putyourheart/article/details/102463731

Tensorflow 2.0 中被移除，最明顯的應該是 tf.contrib 被移除了，而且發生在 1.4，你已經無法使用任何在 contrib module 底下的 code 了。
作為一個開源的深度學習框架，tf.contrib module 一項是社群貢獻 Tensorflow 的結晶，依靠社群的貢獻，有許多實驗性的良好設計，
最後都被收入在 Tensorflow 的官方函式庫中。Tensorflow 在這個 RFC 中提到，因為 contrib 模組愈變愈加龐大，造成維持不易，
最終決定將到 tensorflow addon。

In [ ]:
# tf.layers.dense() -> tf.compat.v1.layers.dense()

In [ ]:
# BERT 源码初探之 run_classifier.py 千次阅读 多人点赞
https://www.csdn.net/tags/MtjaIg0sODQ2MDktYmxvZwO0O0OO0O0O.html

In [ ]:
# tf.nn.dropout -> tf.compat.v1.nn.dropout

# tf.trainable_variables -> tf.compat.v1.trainable_variables 

tf.train.init_from_checkpoint -> tf.compat.v1.train.init_from_checkpoint 

In [ ]:
"""
##########################################################
"""
《命令行参数的模块 - optpars》
https://blog.csdn.net/dcrmg/article/details/79495760


add_option() 加入可用的參數
options，是一个对象（optpars.Values），保存有命令行参数值。
通过命令行参数名，如 file，访问其对应的值： options.file ;


parse_args() 解析終端送來的命令行
args，是一个由 positional arguments 组成的列表;


# add_option()中参数
第1个参数，表示缩写，以单个中划线引导，例如-f、-d，只能用单个字母，可以使用大写;
第2个参数，表示全拼，以两个中划线引导，例如--file、--Opencv_version，可以使用大写;
第1第2个参数，必须保证有其中一个，可以单独使用，也可以同时使用;
从第3个参数开始，是可选参数，是命名参数，常用的有：


type= 表示输入命令行参数的值的类型，默认为string，可以指定为string, int, choice, float，complex其中一种;
default= 表示命令参数的默认值；
dest= 指定参数在options对象中成员的名称，如果没有指定dest参数，将用命令行参数名来对options对象的值进行存取。
help= 显示在帮助文档中的信息;
metavar= 显示到帮助文档中，用来提示用户输入期望的命令参数；


import sys
from optparse import OptionParser
 
# optpars顧名思義，就是由option和parser兩者構建起來
parser = OptionParser() # 創建optpars物件

# 可接收的參數有哪些
parser.add_option('-f','--file',type=str,default='./image',help='file path of images',dest='file_path')
parser.add_option('--weights','-w',type=str,default='./weights_saved',help="file location of the trained network weights")
parser.add_option('--iterations','-i',type=int,default=10000,help='iteration time of CRNN Net')
parser.add_option('--gpu','-g',type=int,default=0,help="gpu id")
 

def main(argv):
    # 解析命令行
    (options, args) = parser.parse_args() 
    (options, args) = parser.parse_args(argv)   # both OK

    print 'file path of images:  ' + options.file_path
    print "file location of the trained network weights:  " + options.weights
    print 'iteration time of CRNN Net:  ' + str(options.iterations)
    print 'gpu id:  ' + str(options.gpu)
 
 
if __name__ == '__main__':
  main(sys.argv)


# 查看帮助文档：
python test.py -h 


# 輸出
Usage: test.py [options]
 
Options:
  -h, --help            show this help message and exit
  -f FILE_PATH, --file=FILE_PATH
                        file path of images
  -w WEIGHTS, --weights=WEIGHTS
                        file location of the trained network weights
  -i ITERATIONS, --iterations=ITERATIONS
                        iteration time of CRNN Net
  -g GPU, --gpu=GPU     gpu id

# 输入命令行参数：
python test.py -f ../tensorflow/train_image -w ../tensorflow/weights -i 5000 -g 2 


file path of images:  ../tensorflow/train_image
file location of the trained network weights:  ../tensorflow/weights
iteration time of CRNN Net:  5000
gpu id:  2

"""
##########################################################
"""
《Python為什麼沒有main函數？為什麼我不推薦寫main函數？》
https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/723146/


C/C++、C#、 Java、Go 和 Rust 等，這些語言是編譯型語言，需要把代碼編譯成可執行的二進制文件，
為了讓操作系統/啟動器找到程序的起點，所以要約定這一個函數。
簡單地說，就是在一大堆代碼裡，需要定義一個顯著的可用於執行的開頭。它們具有特定的含義：


main 函數名是強制的，也就是要求必須有一個 main 函數
main 函數最多隻能有一個，也就是說程序的入口是唯一的
語法格式有一定的要求，具有相對固定的模板


Python，情況就大不相同了。
Python 是解釋型語言，即腳本語言，運行過程是從上往下，逐行解析運行，也就是說它的起點是可知的。
每個 .py 文件就是一個可執行文件，都可作為整個程序的入口文件，也就是說程序的入口是靈活可變的，沒有必須遵守的約定。
有時候運行 Python 項目，並沒有指定入口文件（命令行中較常見，例如”python -m http.server 8000″）。


歸結起來，意思是說 Python 這種腳本語言跟編譯型語言不同，
不像其它語言缺了約定好的入口就沒法執行。


有些知情人要命名出一個”main“函數，其實是想強調它的”主要“地位，想要人為地安排它作為第一個執行的函數。
他們可能認為這樣命名的函數，比較容易記憶。


def main():
…… 
if __name__ == '__main__':
main()


當該文件直接運行時，例如執行
python my_model.py
__name__數值是__main__，因此會執行if內的語句


當該文件以模塊形式導入時，例如執行：
import my_model
__name__數值是my_model，因此不會執行if內的語句


之所以有些知情人要寫if __name__ == '__main__' ，
可能想表明 main() 只有在當前腳本被直接執行時才運行，不希望被導入其它模塊時運行。


明明只有幾十行代碼，或者僅有一個腳本文件，實現一個簡單的功能（一小段爬蟲、用 turtle 畫張圖等等），但是它們都按前面的樣式寫了。


main 入口函數是某些語言特有的，不該在 Python 中“照貓畫虎”。
使用 main.py 而非 main()，建議把入口文件命名為 main.py。


不推薦寫if __name__ == '__main__' 。
首先，如果只有一個文件的話，因為不存在導出的可能，不建議寫。
其次，存在多文件時，入口文件（main.py）中極不推薦寫這一句，
此文件的代碼邏輯應該精煉，理論上其內容不該被導出到其它模塊使用，因為它是起點！


最後，多文件的非入口文件也不建議寫，因為在非入口文件中寫這個判斷，最大的作用就是寫一些測試代碼，
但是測試代碼應該分離出來，寫到專門的目錄或文件中。

"""
##########################################################
"""
《Python 中 -m 的原理》
https://mp.weixin.qq.com/s/co_OAaVQ49rPfgoAh6Pejw


如果“-m”之后要执行的是一个包，那么解释器经过前面提到的查找过程，
先定位到该包，然后会去执行它的“__main__”子模块，也就是说，在包目录下需要实现一个“__main__.py”文件。
换句话说，假设有个包的名称是“pname”，那么，“python -m pname”，其实就等效于“python -m pname.__main__”。


pip，它也是一个包，为什么“python -m pip”的方式可以使用呢？当然是因为它有“__main__.py”文件
# pip包，下有internal、vendor兩個資料夾，以及__init__.py、__main__.py兩個py檔案
“python -m pip”实际上执行的就是这个“__main__.py”文件，它主要作为一个调用入口，调用了核心的"pip._internal.main"。


http 包因为没有一个统一的入口模块，所以采用了“python -m 包.模块”的方式，
而 pip 包因为有统一的入口模块，所以加了一个“__main__.py”文件，最后只需要写“python -m 包”，简明直观。


------------


“python -m pip install xxx”。我们可能会习惯性地使用“pip install xxx”，
或者做了版本区分时用“pip3 install xxx”，总之不在前面用“python -m”做指定。但这种写法可能会出问题。


例如，用“python3.8 -m pip”，可以明确指定给 3.8 版本安装，而不会混淆成其它的版本

"""
##########################################################
"""
《爲什麼要定義main()》
https://kknews.cc/zh-tw/code/gjxb2am.html

儘管Python不要求將函數命名為main()，但是最佳的做法是將入口點函數命名為main()，以便傳達函數的意圖。
即使Python對名為main()的函數並沒有賦予特殊的含義，但這樣方便其他程式設計師定位程序的起點。


In [ ]:
##########################################################
"""
《tensorflow命令行参数：tf.app.flags.FLAGS》
https://blog.csdn.net/dcrmg/article/details/79658725


FLAGS ，用于接受从终端传入的命令行参数，
相当于对Python中的命令行参数模块optpars。
optpars中的参数类型是通过参数 “type=xxx” 定义的。


tf中每个合法类型都有对应的 “DEFINE_xxx”函数。
比如，tf.app.flags.DEFINE_string() ：定义一个用于接收 string 类型数值的变量。
“DEFINE_xxx”函数带3个参数，分别是变量名称，默认值，用法描述例如：
tf.app.flags.DEFINE_string('ckpt_path', 'model/model.ckpt-100000', '''Checkpoint directory to restore''')
定义一个名称是 "ckpt_path" 的变量，默认值是 ckpt_path = 'model/model.ckpt-100000'，描述信息表明这是一个用于保存节点信息的路径。


# -*- coding=utf-8 -*-
 
import tensorflow  as tf
 
FLAGS = tf.app.flags.FLAGS
 
tf.app.flags.DEFINE_string('ckpt_path', 'model/model.ckpt-100000', '''模型保存路径''')
tf.app.flags.DEFINE_float('learning_rate',0.0001,'''初始学习率''')
tf.app.flags.DEFINE_integer('train_steps', 50000, '''总的训练轮数''')
tf.app.flags.DEFINE_boolean('is_use_gpu', False, '''是否使用GPU''')

print '模型保存路径： {}'.format(FLAGS.ckpt_path)
print '初始学习率： {}'.format(FLAGS.learning_rate)
print '总的训练次数： {}'.format(FLAGS.train_steps)
print '是否使用GPU： {}'.format(FLAGS.is_use_gpu)

# 終端機指令
python flags_test.py --ckpt_path abc/cba --learning_rate 0.001 --train_steps 10000 --is_use_gpu True

# 輸出
# FLAGS就相當於是optpars中的option。所以用FLAGS.ckpt_path呼叫,就相當於是呼叫optpars的option.value
模型保存路径： abc/cba
初始学习率： 0.001
总的训练轮数： 10000
是否使用GPU： True

"""
##########################################################

In [ ]:
《tensorflow - tf.app.run()》
https://www.twblogs.net/a/5ccdf3e7bd9eee67a77e7713

tf.app.run() 源碼：
def run(main=None, argv=None):
  """Runs the program with an optional 'main' function and 'argv' list."""

  # Define help flags.
  # 等同於Python的 parser.add_option()
  _define_help_flags()

  # Parse known flags.
  # 等同於Python的 (options, args) = parser.parse_args(argv)
  argv = flags.FLAGS(_sys.argv if argv is None else argv, known_only=True)

  # 等同於Python的 if __name__ == '__main__': main(sys.argv)
  main = main or _sys.modules['__main__'].main

  # Call the main function, passing through any arguments
  # to the final program.
  _sys.exit(main(argv))


# main = main or sys.modules['__main__'].main
正在運行的文件，如果有參數傳入則執行傳入參數，比如，tf.app.run(test)。
代表代碼中的入口函數不叫main()，而是一個其他名字的函數，比如這邊是test()。


如果沒有傳入，比如tf.app.run()，則默認在my_model.py中找main執行，也就是相當於執行tf.app.run(main)，
正在運行的文件，如果沒有傳入參數，而且也沒有main()函數，那run函數就會往上一層去文件中找__main__.py執行。

In [ ]:
《GitHub - uploading an existing file》

https://github.com/phyop/220510_BERT.git
git@github.com:phyop/220510_BERT.git


# …or create a new repository on the command line
echo "# 220510_BERT" >> README.md
git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/phyop/220510_BERT.git
git push -u origin main


# …or push an existing repository from the command line
git remote add origin https://github.com/phyop/220510_BERT.git
git branch -M main
git push -u origin main


# …or import code from another repository
You can initialize this repository with code from a Subversion, Mercurial, or TFS project.


In [ ]:
《GitHub - 生成新的 SSH 密鑰並將其添加到 ssh-agent》
https://docs.github.com/en/authentication/connecting-to-github-with-ssh/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent

https://blog.csdn.net/Sarah_LZ/article/details/119601673

# 生成新的 SSH 密鑰
$ ssh-keygen -t ed25519 -C "your_email@example.com" # 按 Enter。這接受默認文件位置


# 將 SSH 密鑰添加到 ssh-agent
$ eval "$(ssh-agent -s)" # 在後台啟動 ssh-agent
$ ssh-add ~/.ssh/id_ed25519 # 將您的 SSH 私鑰添加到 ssh-agent


《GitHub - 將新的 SSH 密鑰添加到您的 GitHub 帳戶》
https://docs.github.com/en/authentication/connecting-to-github-with-ssh/adding-a-new-ssh-key-to-your-github-account


# 找到隱藏.ssh文件夾，在您喜歡的文本編輯器中打開文件，然後將其複製到剪貼板。
$ cat ~/.ssh/id_ed25519.pub # 將 SSH 公鑰複製到剪貼板


# 在GitHub側邊欄的“setting”部分，單擊“SSH 和 GPG 密鑰”
# 在“標題”字段中，為新鍵添加描述性標籤。例如，如果您使用的是個人 Mac，您可以將此鍵稱為“個人 MacBook Air”。
# 將您的密鑰粘貼到“密鑰”字段中
# 單擊“添加 SSH 密鑰”

In [ ]:
《GitHub - 同步时大小超限问题的解决》

https://banyudu.com/posts/fix-git-pack-exceeds-maximum-allowed-size-problem.b5bac6

在服务端新建了一个空白仓库，重新复现问题。然后执行下面的操作：

添加新的远程仓库：git remote add test-upstream <git仓库地址>
在master分支中执行 git push -u test-upstream master，返回失败信息
基于超大Commit之前的一条Commit，新建分支 tmp
使用 git push -u test-upstream tmp
在tmp分支的基础上，git add 超大分支中的部分文件，然后commit & push
在 tmp 分支上执行 git merge master，合并超大Commit
再次git push -u test-upstream tmp
此时切换到master分支，再将执行 git push -u test-upstream master，成功

In [ ]:
《GitHub - 不再接受帳戶密碼，必須使用token認證》
https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token

https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token

GitHub 會自動刪除一年未使用的個人訪問token

setting->Developer settings -> Personal accesee tokens -> Generate new token -> 打勾repo -> 綠色Generate tokenn

# 獲得token後，您可以在通過 HTTPS 執行 Git 操作時輸入它而不是密碼
# 例如，在命令行中，您將輸入以下內容：
$ git clone https://github.com/username/repo.git
Username: your_username
Password: your_token


In [ ]:
《GitHub - 撤回commit》

# git remote add <name> <your repo url>
git remote add origin-github https://github.com/phyop/220510_BERT.git

# 查git push -u的指令是什麼意思
# -u | --set-upstream
git push --help

# 可能是main，也可能是origin，或是master
# 反正就是一個是指local，一個是指remote
git push -u origin-github main
Username for 'https://github.com': phyop      
Password for 'https://phyop@github.com':Personal access tokens


https://blog.csdn.net/w958796636/article/details/53611133

# 写完代码后，我们一般这样
git add . //添加所有文件
git commit -m "本功能全部完成"


# 执行完commit后，想撤回commit，怎么办？
# 这样就成功的撤销了你的commit
# 注意，仅仅是撤回commit操作，您写的代码仍然保留。
git reset --soft HEAD^


HEAD^的意思是上一个版本，也可以写成HEAD~1
如果你进行了2次commit，想都撤回，可以使用HEAD~2


--mixed 
意思是：不删除工作空间改动代码，撤销commit，并且撤销git add . 操作
这个为默认参数,git reset --mixed HEAD^ 和 git reset HEAD^ 效果是一样的。

--soft  
不删除工作空间改动代码，撤销commit，不撤销git add . 

--hard
删除工作空间改动代码，撤销commit，撤销git add . 
注意完成这个操作后，就恢复到了上一次的commit状态。

--amend
如果commit注释写错了，只是想改一下注释，只需要：
此时会进入默认vim编辑器，修改注释完毕后保存就好了。
git commit --amend


In [ ]:
《GitHub - 清空.git並重建》
https://zhuanlan.zhihu.com/p/355413778

# 先cd到.git的資料夾位置
# 删除舊.git
rm -rf .git

# 在目前位置初始化一個本地repo
git init

# 把本地想同步的文件和repo掛鉤
# 全部都要的話，就用.
git add .

# 把和repo掛鉤的文件，提交目前的版本出去
git commit -m "Initial commit"

# 在本地建立一個分支，叫做main
git branch -M main

# 設定要同步到哪個遠程repo，取name爲origin
# 例如：git remote add origin git@github.com:phyop/220510_BERT.git
git remote add origin git@github.com:<YOUR ACCOUNT>/<YOUR REPOS>.git

# 從本地的main分支，push到遠程的origin分支
git push -u origin main


In [ ]:
《GitHub - Error "Your push would publish a private email address"》
https://stackoverflow.com/questions/43863522/error-your-push-would-publish-a-private-email-address

在 GitHub 的個人設置 → 電子郵件中找到您的 GitHub noreply 地址。在“將我的電子郵件地址保密”複選框的描述中提到了這一點。通常，它以唯一標識符和您的用戶名開頭。

在 Git 中設置電子郵件地址。使用 GitHub 提供的無回復電子郵件地址。

為計算機上的每個存儲庫設置電子郵件地址

git config --global user.email "{ID}+{username}@users.noreply.github.com"
為單個存儲庫設置您的電子郵件地址

git config user.email "{ID}+{username}@users.noreply.github.com"
重置上次提交的作者信息：

git commit --amend --reset-author
如果您的私人電子郵件地址有多個提交，請參閱此答案。

現在您可以使用 noreply 電子郵件地址推送提交，並且未來的提交也將具有 noreply 電子郵件地址。

git push
一旦你配置了 Git，提交將使用你的備用“noreply”電子郵件地址，任何不這樣做的推送都將被拒絕。

# 檢查目前git狀態
git status
git log

In [ ]:
《GitHub - add、commit》
https://w3c.hexschool.com/git/b9be5b1e

Untracked 的意思是，Git 偵測到你有新增這筆檔案，但尚未是 Git 追蹤的對象。
所以必須先將它加入到 staging area(索引)裡，就可以加入到追蹤對象。

# 那麼要如何將檔案加入到索引呢？指令是：
git add <檔案名稱>

# 如果要一口氣將全部檔案加入索引，可以用 
git add .

檔案狀態本來是 Untracked files，變成是Changes to be committed，這樣就表示有成功加入到索引。
Changes to be committed的意思是，放在索引的檔案即將會被提交成一個新版本(commit)。

# 此時我們可以透過以下指令，來提交一個新版本。
git commit -m "<填寫版本資訊>"

# 可以獲得使用資訊
git status

# 輸出會因為你已經提交一個版本(commit)，所以目前工作目錄上已經清空
On branch master
nothing to commit, working tree clean


# 那麼我們要如何觀看新增的版本呢？你可以使用以下指令：
git log

git pull的時候，是同時操作了git fetch與git merge兩項動作

In [ ]:
《GitHub - 删除某个文件》
https://segmentfault.com/a/1190000018921326

在Github上我们只能删除仓库，并不能删除文件或者文件夹，
所以需要单独删除文件或者文件夹只能用命令来解决。

1.本地仓库的文件和远程仓库的文件同时删除
先打开本地仓库的文件夹，选择要删除的文件或者文件夹点击删除，注意git仓库不要删除。
然后执行下面的命令行即可。刷新页面后在看下远程仓库的文件或者文件夹就已经被删除了。

git add *
git commit -m 'del'
git push origin master


2.只删除远程仓库，不删除本地仓库
进入到git bash窗口，执行以下命令：

$ git pull origin master  # 将远程仓库里面的项目拉下来
$ dir # 查看有哪些文件夹
$ git rm -r --cached target  # 删除你要删除的文件名称，这里是删除target文件夹（cached不会把本地的flashview文件夹删除）
$ git commit -m '删除了target' # 提交,添加操作说明
$ git push -u origin master #重新提交（若需要对其他分支进行操作，则把master换为对应分支，如:git push -u origin dev）


In [ ]:
《GitHub - 沒什麼用的實驗過程》
# 要追蹤bert-master的版本更新，所以加入索引
$git add bert-master/

# 如果做一個段落，想要當作一個版本，就做commit
$git commit -m "bert-master"

# 不止在本地，還想push到remote
$git push --set-upstream origin-github main

# 觀看新增的版本
$git log

# 輸出
commit 548d74ffcceddcf4ac328fbba78259341d1f5d2c (HEAD -> main)
Author: phyop <email address>
Date:   Mon May 16 10:45:32 2022 +0800

    run_classifier.py

# hard的話，就是撤回commit，而且code也回復
$git reset --hard HEAD
HEAD is now at 548d74f run_classifier.py

# 獲得使用資訊
$git status

# 輸出
On branch main
nothing to commit, working tree clean

In [ ]:
《Feature()、Example()》
# https://www.tensorflow.org/api_docs/python/tf/train/Feature

import tensorflow as tf
import collections


# 建立一個OrderedDict()物件，來把各種feature裝進去
# 因爲OrderedDict()是一個可以裝東西進去的物件，所以被放在collections類裏面
features = collections.OrderedDict()


# features[將成爲Example中的key名] = tf.train.Feature(int64_list=tf.train.資料形態(value=[實際的list值]))
features["int_ls"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[1, 2, 3, 4]))
# Feature()就是一個單純的dict()
"""
int64_list {
  value: 1
  value: 2
  value: 3
  value: 4
}
"""


# features[keyN名] = tf.train.Feature(valueN名)=tf.train.資料的形態(value=[真正value列表的值]]))
# b'a' 代表把a這個str轉換成byte
features["str_ls"] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'a', b'b']))
"""
bytes_list {
  value: "a"
  value: "b"
}
"""


tf.train.Example(features=tf.train.Features(feature=features))
# Example()是多了一層dict()的複合型字典
"""
features {
  feature {
    key: "int_ls"
    value {
      int64_list {
        value: 1
        value: 2
        value: 3
        value: 4
      }
    }
  }
  feature {
    key: "str_ls"
    value {
      bytes_list {
        value: "a"
        value: "b"
      }
    }
  }
}
"""

'\nfeatures {\n  feature {\n    key: "int_ls"\n    value {\n      int64_list {\n        value: 1\n        value: 2\n        value: 3\n        value: 4\n      }\n    }\n  }\n  feature {\n    key: "str_ls"\n    value {\n      bytes_list {\n        value: "a"\n        value: "b"\n      }\n    }\n  }\n}\n'

In [ ]:
《BERT - 93. 数据讀取、预处理》

《run_classifier.py》

# 數據處理基礎部分的類，也就是這邊要作爲父類使用
# 所以下面有一些子類，會繼承自這個數據處理，去更加完善
class DataProcessor(object):


# 繼承自數據處理基礎的父類DataProcessor
# 換句話說，如果要應用到自己的數據集，就像這裏這樣，仿造着去寫一個子類，繼承自DataProcessor
class MrpcProcessor(DataProcessor):

  def get_labels(self):
    """See base class."""
    # 可以修改自己需要幾個分類，比如可能想要3個等級，那就["0", "1", "2"]
    return ["0", "1"]

  def _create_examples(self, lines, set_type):
      # 比如mrpc一筆data有2句text，但我們的只需要1句，就可以以這邊的code爲基礎去修改，把text_b的部分刪除
      text_a = tokenization.convert_to_unicode(line[3])
      text_b = tokenization.convert_to_unicode(line[4])

def main(_):
  # 設置展示性的logging而已
  tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)


  # 一開始會設置一些if條件式，爲的是去檢查，輸入的參數寫得有沒有問題
  if not FLAGS.do_train and not FLAGS.do_eval and not FLAGS.do_predict:


  # 設置if條件，看有沒有TPU或是GPU之類的，有的話才會起作用的code
  if FLAGS.use_tpu and FLAGS.tpu_name:


  # 要開始train，那就是要開始讀取數據，也就是要開始做數據的處理了
  if FLAGS.do_train:
    # 利用get_train_examples，把數據一行行的讀進來
    # 函數快速定位
    # 點擊get_train_examples，然後壓着ctrl+shift+滑鼠移到get_train_examples字串上
    # 會出現好幾個選項可以跳轉，包括DataProcessor中的def、還有其子類中也有各自的def，
    # 選在class MrpcProcessor(DataProcessor)下的def get_train_example去看。
    train_examples = processor.get_train_examples(FLAGS.data_dir)
    # 3700筆數據,假設batch_size=100,那就是370次iteration,才能完成一個epoch
    num_train_steps = int
        len(train_examples) / FLAGS.train_batch_size * FLAGS.num_train_epochs)
    # 比如總共迭代1375次,那一開始就用比較小的學習率來warmup，直到過了1375*0.1次迭代，才恢復到正常的學習率。
    num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)


  # 先把一些需要用到的物件實例化出來，實際要怎麼用、引數的值爲什麼是這樣，要往後面的code繼續看
  model_fn = model_fn_builder(...)


  if FLAGS.do_train:
    train_file = os.path.join(FLAGS.output_dir, "train.tf_record")
    # 在這邊打下斷點，也就是「掛起」
    # 然後用「debug調試」模式，如果前面也有別的斷點，那就點擊「步進/恢復程序」直到code執行到這裏
    # 在「布局設置」中，把「變量」給打勾顯示出來，就可以查找到Name-Value，比如「num_train_steps={int}1375」
    # ctrl+shift跳進去file_based_convert_examples_to_features()查看源碼
    file_based_convert_examples_to_features(
        train_examples, label_list, FLAGS.max_seq_length, tokenizer, train_file)  


def file_based_convert_examples_to_features(
    examples, label_list, max_seq_length, tokenizer, output_file):
  """Convert a set of `InputExample`s to a TFRecord file."""

  # writer = tf.python_io.TFRecordWriter(output_file)
  # 這邊也打下斷點，看看人家io是怎麼做的
  # 對數據處理好特徵之後，轉換成TFRecord的格式，讀數據快，處理也方便
  # 等下會遍歷每一個樣本，用TFRecordWriter去寫入到TFRecord當中
  # 接下來就不用到原始數據中去讀了，直接去TFRecord讀取處理好的特徵就行了
  writer = tf.compat.v1.python_io.TFRecordWriter(output_file)

  for (ex_index, example) in enumerate(examples):
    # 每隔1萬次，打印一次結果。因爲mrpc只有3700筆，所以這邊是打印不出來了
    if ex_index % 10000 == 0:
      # tf.compat.v1.logging.info("Writing example %d of %d" % (ex_index, len(examples)))
      tf.compat.v1.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    # 一樣，convert_single_example看不懂那就跳進去源碼看看
    feature = convert_single_example(ex_index, example, label_list,
                                     max_seq_length, tokenizer)


In [ ]:
《BERT - 94. tfrecord制作》

def convert_single_example(ex_index, example, label_list, max_seq_length,
                           tokenizer):
  ...

  label_map = {}
  for (i, label) in enumerate(label_list):
    # 設置斷點，然後步進，就可以去布局設置中的變量，查找到label_map={'0': 0, '1': 1}
    # 所以知道就只是設置個標籤而已，沒什麼特別
    label_map[label] = i

  # 跳進tokenize()查看源碼
  tokens_a = tokenizer.tokenize(example.text_a) # 對一筆數據中的第1句話做分詞
  tokens_b = None # 默認是沒有第二句話
  if example.text_b: # 這邊會做判斷，看數據是不是不止有1幾句話組成
    tokens_b = tokenizer.tokenize(example.text_b) # 對一筆數據中的第2句話做分詞

  if tokens_b:
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    # 一筆數據2句話：[CLS], 序列窗口1, [SEP], 序列窗口2, [SEP]
    # 有b的時候，保留3個特殊字符
    # 一筆資料中的第一句話，開頭會有[CLS]特殊字符，是待分類特徵，看是0還是1
    # 第一句話的結束、第2句話的結束，各會插入一個特殊字符[SEP]

    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3) # 如果這兩個太長就截斷
  else:
    # Account for [CLS] and [SEP] with "- 2"
    # 一筆數據1句話：[CLS], 序列窗口1, [SEP]
    # 沒有b的時候，特殊字符保留2個就夠了
    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[0:(max_seq_length - 2)]

  # The convention in BERT is:
  # (a) For sequence pairs:
  # 一筆數據2句話：[CLS], 序列窗口1, [SEP], 序列窗口2, [SEP]
  # 第一句話：[CLS] is this jack ##son ##ville ? [SEP]
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  # type_ids只有0和1兩種值，一堆0的代表描述前面那句話，一堆1的代表描述代表後面那句
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  tokens = [] # 詞
  segment_ids = [] # 編碼這個詞是來自於第幾句
  tokens.append("[CLS]") # 一筆數據不管是1句話還是2句話，都是[CLS]開始, [SEP]結尾
  segment_ids.append(0) # [CLS]的編碼就是0
  for token in tokens_a: # 遍歷a，也就是第一句話，當中的每個詞
    tokens.append(token) # 把第一句話的每個詞加進來list
    segment_ids.append(0) # 因爲是第一句話，所以ids都是0；如果是第2句話的話，那ids就都是1了
  tokens.append("[SEP]")
  segment_ids.append(0) # 第一句話的結束符，當然也是第一句話的一部分，所以ids也是0

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)

  # BERT开源项目及数据/GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/vocab.txt
  # 語料表中的第幾行，就代表詞索引ID，所以語料表也就是詞ID表
  # convert_tokens_to_ids()就是把詞轉成ID的映射函數
  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  # 時序窗格大小必須固定，就如同conv的大小是固定的，因爲這樣特徵的shape才會固定，才不會因爲不匹配而無法運算
  # 也就是說，如果2個句子加起來還不到max_seq_length的長度，那就要做pad補齊
  # 但那就有一個問題啦，後面補齊的那些，根本不需要attention
  # 所以才做一個mask來，把所有實際上有的詞（input_ids），全部都用元素1來做標記
  # 後續執行到self-attention的時候，只去對mask值爲1的地方做計算
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length: # 如果2個句子加起來還不到max_seq_length的長度，那就要做pad補齊
    # input_ids -> {list:128}[101, 2572, ...,1012, 102, 0, 0,..., 0, 0]
    input_ids.append(0) # pad的詞id就是0
    input_mask.append(0) # pad不需要被計算self-attention，所以mask值爲0
    # pad被算在第1或2句話都不對，所以就乾脆也給0的值就好
    # 此外，在第2句話結束，要轉成pad的時候，有一個從一堆1轉成一堆0的轉折，print查錯也比較容易看出來
    # segment_ids -> {list:128}[0, 0,...0, 0, 1, 1, ...,1, 1, 0, 0, ..., 0, 0]
    segment_ids.append(0)


  assert len(input_ids) == max_seq_length # 用assert來寫一些簡單判斷，比如等於時序窗格長度，確定程式執行結果沒有基本上的出錯
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  label_id = label_map[example.label]
  if ex_index < 5:
    tf.compat.v1.logging.info("*** Example ***")
    tf.compat.v1.logging.info("guid: %s" % (example.guid))
    # I0515 00:08:47.119734 140607478482752 run_classifier.py:500] tokens: [CLS] am ##ro ##zi accused his brother , whom he called " the witness " , of deliberately di ##stor ##ting his evidence . [SEP] referring to him as only " the witness " , am ##ro ##zi accused his brother of deliberately di ##stor ##ting his evidence . [SEP]
    # 5/15 凌晨0點8分47秒119734微秒 時間戳 檔名.py:第500行程式碼] tokens: [CLS] 第1句話的分詞 [SEP] 第2句話的分詞 [SEP]
    tf.compat.v1.logging.info("tokens: %s" % " ".join( # 分詞
        [tokenization.printable_text(x) for x in tokens]))
    # 時序窗格包含了特殊符號以及2句話，特殊符號也是有自己的詞id，而input_ids是把時序窗格轉成詞id的結果
    # input_ids: 101 2572 ... 1012 102 0 ... 0
    tf.compat.v1.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids])) # 詞id
    tf.compat.v1.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask])) # pad是0,非pad是1
    tf.compat.v1.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids])) # 第1句是0,第2句是1
    tf.compat.v1.logging.info("label: %s (id = %d)" % (example.label, label_id)) # 就是[cls]，不是0就是1

  # 進去InputFeatures()源碼看看是什麼作用
  # 反正就是把一堆上面會用到的值，建立成屬性：self.input_ids、self.input_mask、self.segment_ids、self.label_id、self.is_real_example
  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      label_id=label_id,
      is_real_example=True)
  return feature



def file_based_convert_examples_to_features(
    examples, label_list, max_seq_length, tokenizer, output_file):
  """Convert a set of `InputExample`s to a TFRecord file."""

  # writer = tf.python_io.TFRecordWriter(output_file)
  # 這邊也打下斷點，看看人家io是怎麼做的
  # 對數據處理好特徵之後，轉換成TFRecord的格式，讀數據快，處理也方便
  # 等下會遍歷每一個樣本，用TFRecordWriter去寫入到TFRecord當中
  # 接下來就不用到原始數據中去讀了，直接去TFRecord讀取處理好的特徵就行了
  writer = tf.compat.v1.python_io.TFRecordWriter(output_file)

  for (ex_index, example) in enumerate(examples): # 遍歷每一個樣本，而mrpc有3700個訓練樣本
    # 每隔1萬次，打印一次結果。因爲mrpc只有3700筆，所以這邊是打印不出來了
    if ex_index % 10000 == 0:
      # tf.compat.v1.logging.info("Writing example %d of %d" % (ex_index, len(examples)))
      tf.compat.v1.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    # 一樣，convert_single_example看不懂那就跳進去源碼看看
    # 傳入for中的3700個樣本們examples，得到其中的一個樣本example的feature
    feature = convert_single_example(ex_index, example, label_list,
                                     max_seq_length, tokenizer)

    # 雖然GPU訓練快，但是讀取數據這一步慢了，为了可以高效地读取数据，我們先將data整理成可訓練的TFRecord序列狀態
    # 首先將各項特征feature，都包進去tf.Example，数据會被表示为{"string": value}形式，再去使用tf.string做成序列化來儲存
    # 《TFRecord步驟》 tf.train.Feature() -> tf.train.Example() -> tf.string序列化
    # 《Feature》先爲自己需要的資料，創建對應的feature型別，指派給一個變數名稱
    # 例如int_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[1, 2, 3, 4]))
    # 變數名稱就是指 "tf.train.Example(features=tf.train.Features(feature={}))" 中的value
    # 創建一個存儲訓練數據的標準原型 tf.train.Example()
    # Example()裏面放的是tf.train.Features()，用features當參數給值（有s）
    # Features()裏面放的是{feature={'string':value, ...}，用feature當參數給值（沒s）
    # 例如：
    # example = tf.train.Example(
    #     features=tf.train.Features(feature={
    #         'my_ints': int_feature,
    #         'my_floats': float_feature,
    #         'my_bytes': bytes_feature,
    #     }))
    # Example.features儲存鍵值對，其中每個鍵（字符串）映射到tf.train.Feature，包含固定的列表訊息
    # 這種靈活緊湊的格式，允許存儲大量的類型化數據（詳見 tf.io.parse_example）
    # https://www.tensorflow.org/tutorials/load_data/tfrecord
    # _int64_feature() 之類的方法，是爲了給內部使用的，比如把資料轉型成 tf.train.Example 需要的格式
    # 反正就是TF定義的標準API，幫忙user構建數據集，不是給外部呼叫，所以用底線開頭的私有方法
    # 所以user有數據，往裏面套用就對了，這種底線開頭的，不用研究的太過深入。比如：
    # 呼叫函數_int64_feature，會返回一個類似字典的結構
    # >>> print(_int64_feature(1) # 輸出 int64_list {value: 1}
    # 告訴我們裏面裝的類型是int64_list，值是1
    # 特徵就不會至於單一種類型而已，轉什麼類型還是要自己決定。例如
    # feature = {
    #         'feature0': _int64_feature(feature0), # 把feature0轉成int64的格式
    #         'feature2': _bytes_feature(feature1), # 把feature1轉成bytes的格式
    #         'feature3': _float_feature(feature2), # 把feature2轉成float的格式
    #       }
    # 第1個feature是Features的參數，第2個feature是引數（也就是轉換完的結果）
    # example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    # 上面code代表，有了各項特征之後，就可以包在一起，存成Example
    # Example -> features{feature字典1, ..., feature字典n}
    # example_proto.SerializeToString() # 將Example转换为二进制字符串

    def create_int_feature(values):
      f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return f

    # 建立一個OrderedDict()物件，來把各種feature裝進去
    # 因爲OrderedDict()是一個可以裝東西進去的物件，所以被放在collections類裏面
    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_int_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)
    features["label_ids"] = create_int_feature([feature.label_id])
    features["is_real_example"] = create_int_feature(
        [int(feature.is_real_example)])

    # 最後面()的第1個feature是Features的參數，第2個feature是引數（也就是轉換完的結果）
    # 把各種feature用OrderedDict()打包起來，然後傳給Example()，再去使用tf.string做成序列化來儲存
    # Feature()就是一個單純的dict()
    # Example()是多了一層dict()的複合型字典
    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    # 雖然GPU訓練快，但是讀取數據這一步慢了，为了可以高效地读取数据，我們先將data整理成可訓練的TFRecord序列狀態
    # 《TFRecord步驟》 tf.train.Feature() -> tf.train.Example() -> tf.string序列化
    writer.write(tf_example.SerializeToString())
    # 到目前爲止，只是拿出其中一筆樣本來做成序列化而已，而mrpc有3700個訓練樣本，所以外層有個for去遍歷
  writer.close()

In [ ]:
《BERT - 95. Embedding层的作用》
# 跳進去BertModel()看看源碼
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  model = modeling.BertModel( # BertModel裏面包含了有transformer
      config=bert_config, # 反正就是一下類似字典的各種參數配置
      is_training=is_training,
      input_ids=input_ids, # 詞向量(batch_size=8, 時序窗格長度=128)
      input_mask=input_mask, # pad是0,非pad是1。執行到self-attention的時候，只去對mask值爲1的地方做計算
      token_type_ids=segment_ids, # 第1句話是0,第2句話是1
      use_one_hot_embeddings=use_one_hot_embeddings) # 用TPU的時候，才會考慮one_hot，所以use_one_hot_embeddings默認是False


《modeling.py》
class BertModel(object):
  ...
  def __init__(self,
    ...
    input_shape = get_shape_list(input_ids, expected_rank=2) # (batch_size=8, 時序窗格長度=128)
    batch_size = input_shape[0]
    seq_length = input_shape[1]

    if input_mask is None:
      # 如果自己沒設mask，那就是全都要用的意思，所以也就相當有給mask，然後全部的值都給1
      input_mask = tf.ones(shape=[batch_size, seq_length], dtype=tf.int32)

    if token_type_ids is None:
      # 自己沒設詞向量，那就相當於有給詞向量，然後全部的值都給0
      token_type_ids = tf.zeros(shape=[batch_size, seq_length], dtype=tf.int32)

    # with tf.compat.v1.variable_scope(scope, default_name="bert"):
    with tf.compat.v1.variable_scope(scope, default_name="bert"):
      with tf.compat.v1.variable_scope("embeddings"):
        # Perform embedding lookup on the word ids.
        (self.embedding_output, self.embedding_table) = embedding_lookup(
            # 這邊的embedding詞嵌入有3種
            # 輸入ids、輸入遮罩、詞位置ids
            # input_ids=input_ids, # 詞輸入(batch_size=8, 時序窗格長度=128)
            # input_mask=input_mask, # pad是0,非pad是1。執行到self-attention的時候，只去對mask值爲1的地方做計算
            # token_type_ids=segment_ids, # 第1句話是0,第2句話是1
            # 如果想要把所有embedding融合成一個，那上面3個embedding的維度就必須要一致
            input_ids=input_ids, # 詞輸入(batch_size=8, 時序窗格長度=128)
            # 預訓練模型中的詞大表，共有30522個詞，在txt檔案當中
            # GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/vocab.txt
            vocab_size=config.vocab_size,
            # 詞映射成多少維、判斷2個句子是不是同一個意思、...，但凡要加入的訊息，都加入到這768維度裏面
            # 用人家的模型，就要用人家的維度，去改shape的話，會造成無法使用人家的預訓練結果
            # 768維預訓練的結果，就在這個440.4MB大容量的.ckpt檔案中
            # GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
            embedding_size=config.hidden_size,
            initializer_range=config.initializer_range, # 初始化的起始範圍
            word_embedding_name="word_embeddings",
            use_one_hot_embeddings=use_one_hot_embeddings) # 默認是False，也就是沒TPU的情況下
            

In [ ]:
《BERT - 96. 加入额外编码特征》

def func(參數1,
         參數2,):
  """ 說明這個函數是幹嘛用的
  空一行
    Args:
        參數1:意義說明
        參數2:意義說明
    Returns:
        返回形態，返回值的意義
  """
  # 使用注意1
  # 使用注意2


# 觀察輸入、輸出的shape，可以增加對函數的理解性
# In: [batch_size, seq_length] -> (8, 128) 
# Out: [batch_size, seq_length, embedding_size] -> (8, 128, 764)
def embedding_lookup(input_ids,
  ...
  if input_ids.shape.ndims == 2:
    # 這邊顯示的資料形態是{Tensor:(4, 128, 1)}，但唐影片顯示的是(8, 128, 1)
    # 因爲我在"運行配置"的"形參"，--train_batch_size設定的是4
    # "--task_name=MRPC \ --do_train=true \ --do_eval=true \ --data_dir=../GLUE/glue_data/MRPC \ --vocab_file=../GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/vocab.txt \ --bert_config_file=../GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/bert_config.json \ --init_checkpoint=../GLUE/BERT_BASE_DIR/uncased_L-12_H-768_A-12/bert_model.ckpt \ --max_seq_length=128 \ --train_batch_size=4 \ --learning_rate=2e-5 \ --num_train_epochs=3.0 \ --output_dir=../GLUE/output/"
    # 因爲我的筆電設定8的時候，batch就會顯示None。數值往下實驗，最多跑到batch_size=7,還可以顯示的出來，9也顯示不出來，所以乾脆就設個4就好
    # 把input_ids的維度，從(8, 128)多expand一個維度到(8, 128, 1)，方便做lookup
    input_ids = tf.expand_dims(input_ids, axis=[-1])

  # embedding_table就是詞映射矩陣，(30522, 768)，每一個詞都是一個768的維度
  embedding_table = tf.compat.v1.get_variable(
      name=word_embedding_name,
      shape=[vocab_size, embedding_size], # (30522, 768)，每一個詞都是一個768的維度
      initializer=create_initializer(initializer_range))

  flat_input_ids = tf.reshape(input_ids, [-1])
  if use_one_hot_embeddings: # 沒有TPU，所以不做one-hot，所以直接跳過這個if，看到下面的else
    one_hot_input_ids = tf.one_hot(flat_input_ids, depth=vocab_size)
    output = tf.matmul(one_hot_input_ids, embedding_table)
  else:
    # 一個batch有8個樣本，每個樣本有128個詞。所以一個batch要查找1024個詞
    # output的shape是(1024, 768)，也就是對這一個batch的1024個詞，全部都查找得到768個維度的詞向量映射了
    output = tf.gather(embedding_table, flat_input_ids)

  input_shape = get_shape_list(input_ids) # (8, 128, 768)

  output = tf.reshape(output,
                      input_shape[0:-1] + [input_shape[-1] * embedding_size])
  return (output, embedding_table)


In [ ]:
《96. 加入额外编码特征》
# 把位置編碼給加進去，但shape不變
# https://zhuanlan.zhihu.com/p/152253628
def embedding_postprocessor(input_tensor,
  ...
  
  input_shape = get_shape_list(input_tensor, expected_rank=3) # (8, 128, 768)
  ...
  output = input_tensor # 只是加入位置編碼訊息，不改變shape

  if use_token_type:
    if token_type_ids is None:
      raise ValueError("`token_type_ids` must be specified if"
                       "`use_token_type` is True.")
    # 之前詞向量是因爲有30522個值，所以維度是(30522, 768)
    # 同理，token_type只有第一句話或第二句話，值就是0/1兩種變化，所以維度是(2, 768)
    token_type_table = tf.compat.v1.get_variable(
        name=token_type_embedding_name,
        shape=[token_type_vocab_size, width],
        initializer=create_initializer(initializer_range))
    # 一共有8×128=512個詞，(batch_size, window_len)，所以flat_token_type_ids.shape=(512,), dtype=int32
    # 對這512個詞都知道了它們各別的句子位置是0還是1
    flat_token_type_ids = tf.reshape(token_type_ids, [-1]) #
    # 論文中說，one_hot是爲了做加速。比如句子1的編碼是10,句子2的編碼是01之類的
    one_hot_ids = tf.one_hot(flat_token_type_ids, depth=token_type_vocab_size)
    # one_hot_ids是(batch中有1024詞, 詞位置有2種可能性)；token_type_table是(詞位置有2種可能性, 詞向量768維)
    # 所以相乘之後的token_type_embeddings，是(batch中有1024詞, 詞向量768維)
    token_type_embeddings = tf.matmul(one_hot_ids, token_type_table)
    # 做個reshape之後，就和原本的input_shape (8, 128, 768)一致了
    # 代表沒有改變形狀，但是過程中把詞位置的資訊增加進去了
    token_type_embeddings = tf.reshape(token_type_embeddings,
                                       [batch_size, seq_length, width])
    output += token_type_embeddings

《97. 加入位置编码特征》

  # 默認是True，也就是需要加上位置編碼訊息
  if use_position_embeddings:
    # assert_op = tf.assert_less_equal(seq_length, max_position_embeddings)
    assert_op = tf.compat.v1.assert_less_equal(seq_length, max_position_embeddings)
    with tf.control_dependencies([assert_op]):
      # 初始化限制位置最多有512個，而每個位置有768維度的向量
      # 這個512和batch_size沒關系
      full_position_embeddings = tf.compat.v1.get_variable(
          name=position_embedding_name,
          shape=[max_position_embeddings, width],
          # 根据给定范围创建截尾正态分布初始器
          initializer=create_initializer(initializer_range))
      ...
      # 也就是只對seq_length的長度去切片進行編碼，原本初始化的512個位置剩下一大堆用不到的就不編碼了
      # position_embeddings是(128, 768)
      position_embeddings = tf.slice(full_position_embeddings, [0, 0],
                                     [seq_length, -1])
      num_dims = len(output.shape.as_list())
      # 只有最後兩個維度是相關的（`seq_length` 和 `width`），所以我們在第一個維度中廣播，這通常只是批量大小。
      position_broadcast_shape = []
      for _ in range(num_dims - 2):
        position_broadcast_shape.append(1)
      # 對batch當中的每一筆數據，都把[128, 768]的位置訊息加進去
      # 位置編碼和詞無關，也和batch中的第幾筆數據無關，只和在什麼位置有關而已，當然也和batch無關
      # 所以這邊做的是，要把原始input_shape[8, 128, 768]中的batch關聯性給拿掉
      position_broadcast_shape.extend([seq_length, width])
      # 然後以[1, 128, 768] 來表示
      position_embeddings = tf.reshape(position_embeddings,
                                       position_broadcast_shape)
      # 最後再把這些位置編碼訊息，做成一張embedding大表
      # 所以最後的embedding大表，就融合了3種嵌入
      # 原本的詞向量、在token_type_ids第0/1個句子、128個位置的哪個位置
      output += position_embeddings

  # 這邊也是一層，是embedding_lookup層，而layer可以去做normalization和dropout
  # 把這層的layer張量值進行標準化，然後再丟進去做dropout來提高模型的泛化能力
  output = layer_norm_and_dropout(output, dropout_prob)
  return output